In [1]:
import pandas as pd
import numpy as np
import random
from ortools.sat.python import cp_model

In [2]:
def doub_row(row):
    hora_inicial = pd.to_datetime(row['Horário'], format='%H:%M:%S')
    
    nova_linha_1 = row.copy()
    nova_linha_2 = row.copy()
    
    nova_linha_1['Horário'] = (hora_inicial).strftime('%H:%M:%S')
    nova_linha_2['Horário'] = (hora_inicial + pd.Timedelta(hours=1)).strftime('%H:%M:%S')
    
    return pd.DataFrame([nova_linha_1, nova_linha_2])

def tri_row(row):
    hora_inicial = pd.to_datetime(row['Horário'], format='%H:%M:%S')
    
    nova_linha_1 = row.copy()
    nova_linha_2 = row.copy()
    nova_linha_3 = row.copy()
    
    nova_linha_1['Horário'] = (hora_inicial).strftime('%H:%M:%S')
    nova_linha_2['Horário'] = (hora_inicial + pd.Timedelta(hours=1)).strftime('%H:%M:%S')
    nova_linha_3['Horário'] = (hora_inicial + pd.Timedelta(hours=2)).strftime('%H:%M:%S')
    
    return pd.DataFrame([nova_linha_1, nova_linha_2, nova_linha_3])

def desaninhar_dias(df):
    df['Dias da Semana'] = df['Dias da Semana'].str.split(',')
    df = df.explode('Dias da Semana')
    return df

# Aplicar a função para todas as linhas triplas
def apply_func(df,func):
    linhas_triplicadas = df.apply(func, axis=1)
    return pd.concat(linhas_triplicadas.tolist(), ignore_index=True)

In [3]:
aulas_raw = pd.read_excel('/home/cayena/Downloads/ROTA TESTE CARLOS E ALLAN.xlsx',header=1)
aulas = aulas_raw.loc[~aulas_raw['Grupo'].isnull()][['Grupo','Horário','Dias da Semana']]

In [4]:
aulas = aulas.loc[aulas['Dias da Semana']!='FOLDER'].reset_index(drop=True)

In [5]:
aulas['Dias da Semana'] = aulas['Dias da Semana'].str.replace('EVERYDAY','2ª ● 3ª ● 4ª ● 5ª ● 6ª')

In [6]:
tri = aulas[aulas['Dias da Semana'] == 'Saturday - Triple'].copy()
doub = aulas[aulas['Dias da Semana'].str.contains('DOUBLE')].copy()
aulas_simples = aulas[~aulas['Dias da Semana'].str.contains('DOUBLE|Saturday - Triple')].copy()

In [7]:
# tratando os dados para colocar cada linha uma aula
aulas['Dias da Semana'] = aulas['Dias da Semana'].str.replace('EVERYDAY','2ª ● 3ª ● 4ª ● 5ª ● 6ª')

# separando as aulas que são triplas, duplas e o resto
tri = aulas.loc[aulas['Dias da Semana'] == 'Saturday - Triple']
doub = aulas.loc[aulas['Dias da Semana'].str.contains('DOUBLE')]
aulas_simples = aulas[~aulas['Dias da Semana'].str.contains('DOUBLE|Saturday - Triple')].copy()

# tratando a colunas horario
aulas_simples['Horário'] = pd.to_datetime(aulas_simples['Horário'],format='%H:%M:%S').dt.strftime('%H:%M:%S')

In [8]:
# transformando aulas duplas/triplas em 2/3 linhas
aulas_duplicadas = apply_func(doub, doub_row)
aulas_triplicadas = apply_func(tri, tri_row)

# padronizando a coluna de dias da semana
aulas_duplicadas['Dias da Semana'] = aulas_duplicadas['Dias da Semana'].str.replace('DOUBLE','')
aulas_triplicadas['Dias da Semana'] = aulas_triplicadas['Dias da Semana'].str.replace(' - Triple','')

# juntando todas as linhas
df_tratado = pd.concat([aulas_simples, aulas_duplicadas, aulas_triplicadas], ignore_index=True)

# desaninhando os dias da semana
df_tratado['Dias da Semana'] = df_tratado['Dias da Semana'].str.replace('●',',')
df_tratado['Dias da Semana'] = df_tratado['Dias da Semana'].str.split(',')
df_tratado = df_tratado.explode('Dias da Semana').reset_index(drop=True)

In [11]:
# Definindo nomes dos professores
Professores = {'Name':['Allan','Carlos','Henrique','Yasmin','Malu','Tanze',
                       'Okamoto','Edu','Wolf','Odin','Renan','Gabiraes',
                       'Nicolas','Novaes','Jorge','Kaori','Igor','Magnus',
                       'Nepo','Jurema','Zoe','Humberto','Luiz','Ricardo','Ian']}
teachers = pd.DataFrame(Professores)
teachers.shape


(25, 1)

In [59]:
all_teacher = teachers['Name'].to_list()
# Dicionário de alocações
alocacoes = {}

# Modelo
model = cp_model.CpModel()

for i in all_teacher:
    for _, row in df_tratado.iterrows():
        g = row['Grupo']
        h = row['Horário']
        d = row['Dias da Semana']
        alocacoes[(i,g,h,d)] = model.NewBoolVar(f"{i}_converinglesson_{g}_period_{h}_day_{d}")

# Restrições 1 
# Um professor nao pode dar aula em dois horarios iguais no mesmo dia
for i in all_teacher:
    for d in df_tratado['Dias da Semana'].unique():
        for h in df_tratado['Horário'].unique():
            chaves_validas = [(i,g,h,d) for i in all_teacher if (i,g,h,d) in alocacoes]
            if chaves_validas:
                model.Add(sum(alocacoes[(i,g,h,d)] for g in df_tratado['Grupo'].unique()) <= 1)

#Solução
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print("Solução encontrada!")
    
    # Imprimir as alocações
    for i in all_teacher:
        for _, row in df_tratado.iterrows():
            g = row['Grupo']
            h = row['Horário']
            d = row['Dias da Semana']
            if solver.BooleanValue(alocacoes[(i,g,h,d)]):
                print(f"{i} está alocado para {g} no horário {h} no dia {d}")
else:
    print("Nenhuma solução viável foi encontrada.")

KeyError: ('Allan', 'MBA DAVI BRAGA', '08:00:00', 'Saturday')

In [52]:
for _, row in df_tratado.iterrows():
        g = row['Grupo']
        h = row['Horário']
        d = row['Dias da Semana']
        model.AddExactlyOne(alocacoes[(i,g,h,d)] for i in all_teacher)

In [55]:
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print("Solução encontrada!")
    
    # Imprimir as alocações
    for i in all_teacher:
        for _, row in df_tratado.iterrows():
            g = row['Grupo']
            h = row['Horário']
            d = row['Dias da Semana']
            if solver.BooleanValue(alocacoes[(i,g,h,d)]):
                print(f"{i} está alocado para {g} no horário {h} no dia {d}")
else:
    print("Nenhuma solução viável foi encontrada.")

Solução encontrada!
Nicolas está alocado para MBA DAVI BRAGA no horário 14:00:00 no dia 2ª
Nicolas está alocado para MBA LIGA DOS CAMPEÕES no horário 18:30:00 no dia 5ª
Nicolas está alocado para MBA IDEIAS EM AÇÃO no horário 19:00:00 no dia 2ª
Nicolas está alocado para GREENSBURG ONLINE no horário 07:00:00 no dia 2ª 
Nicolas está alocado para GREENSBURG ONLINE no horário 07:00:00 no dia  3ª 
Nicolas está alocado para GREENSBURG ONLINE no horário 07:00:00 no dia  4ª 
Nicolas está alocado para GREENSBURG ONLINE no horário 07:00:00 no dia  5ª
Nicolas está alocado para SAVANAH PRESENCIAL no horário 07:00:00 no dia 2ª 
Nicolas está alocado para SAVANAH PRESENCIAL no horário 07:00:00 no dia  3ª 
Nicolas está alocado para SAVANAH PRESENCIAL no horário 07:00:00 no dia  4ª 
Nicolas está alocado para SAVANAH PRESENCIAL no horário 07:00:00 no dia  5ª
Nicolas está alocado para CONVERSATION 14 PRESENCIAL no horário 08:00:00 no dia 3ª
Nicolas está alocado para FERNANDO CAJE - PRESENCIAL no horário 0

In [ ]:
#codição : Cada turma só pode ter 1 professor
for _, row in df_tratado.iterrows():
    g = row['Grupo']
    h = row['Horário']
    d = row['Dias da Semana']
    model.Add(sum(alocacoes[(i,g,h,d)] for i in all_teacher) == 1)

for i in all_teacher:
    for horario in df_tratado['Horário'].unique():
        model.Add(sum(alocacoes[(i, row['Grupo'],horario,row['Dias da Semana'])] for _, row in df_tratado.iterrows() if row['Horário'] == horario) <= 1)

In [34]:
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print("Solução encontrada!")
    
    # Imprimir as alocações
    for professor in all_teacher:
        for _, row in df_tratado.iterrows():
            turma = row['Grupo']
            horario = row['Horário']
            if solver.BooleanValue(alocacoes[(professor, turma, horario)]):
                print(f"{professor} está alocado para {turma} no horário {horario}")
else:
    print("Nenhuma solução viável foi encontrada.")

Nenhuma solução viável foi encontrada.


In [16]:
len(alocacoes)

10650

In [40]:
model = cp_model.CpModel()

In [ ]:
model.

In [ ]:
for a in all_group:
    b = df_tratado.loc[df_tratado['Grupo'] == a, 'Horário'].values[0]
    model.Add(sum(alocacoes[(c,a,b)] for c in all_teacher) == 1)

for professor in all_teacher:
    for horario in df_tratado['Horário'].unique():
        model.Add(sum(alocacoes[(professor, row['Grupo'], horario)] for _, row in df_tratado.iterrows() if row['Horário'] == horario) <= 1)

In [ ]:
model = cp_model.CpModel()

# Dicionário para armazenar as variáveis booleanas
alocacoes = {}

# Criar variáveis booleanas para as combinações de turma e horário existentes
for professor in all_teacher:
    for _, row in aulas_segunda.iterrows():
        turma = row['Grupo']
        horario = row['Horário']
        
        nome_var = f'{professor}_{turma}_{horario}'
        alocacoes[(professor, turma, horario)] = model.NewBoolVar(nome_var)

# Adicionar a restrição: um professor não pode estar em duas turmas no mesmo horário
for _, row in aulas_segunda.iterrows():
    turma = row['Grupo']
    horario = row['Horário']
    model.Add(sum(alocacoes[(professor, turma, horario)] for professor in all_teacher) == 1)

# Nova restrição: um professor só pode estar em uma turma por horário
for professor in all_teacher:
    for horario in aulas_segunda['Horário'].unique():
        model.Add(sum(alocacoes[(professor, row['Grupo'], horario)] for _, row in aulas_segunda.iterrows() if row['Horário'] == horario) <= 1)


# Criar o solver e rodar o modelo

solver = cp_model.CpSolver()
status = solver.Solve(model)
# Imprimir as alocações
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print("Solução encontrada!")
    
    # Imprimir as alocações
    for professor in all_teacher:
        for _, row in aulas_segunda.iterrows():
            turma = row['Grupo']
            horario = row['Horário']# Nova restrição: um professor só pode estar em uma turma por horário
            # Verificar se o professor foi alocado para esta turma no horário específico
            if solver.BooleanValue(alocacoes[(professor, turma, horario)]):
                print(f"{professor} está alocado para {turma} no horário {horario}")
else:
    print("Nenhuma solução viável foi encontrada.")

TESTE

In [41]:
model = cp_model.CpModel()
bonus = model.NewIntVar(0, 1, 'bonus')

In [42]:
bonus

bonus(0..1)

In [36]:
from ortools.sat.python import cp_model

def alocar_professores(turmas, horarios, professores):
    # Inicializar o modelo
    modelo = cp_model.CpModel()

    # Variáveis de decisão: se um professor é atribuído a uma turma em um horário específico
    alocacao = {}
    for t in range(len(turmas)):
        for p in range(len(professores)):
            alocacao[(t, p)] = modelo.NewBoolVar(f'turma_{t}_professor_{p}')

    # Restrição 1: Cada turma deve ter exatamente um professor
    for t in range(len(turmas)):
        modelo.AddExactlyOne(alocacao[(t, p)] for p in range(len(professores)))

    # Restrição 2: Cada professor pode ser alocado a no máximo uma turma por horário
    for p in range(len(professores)):
        for h in set(horarios):
            modelo.AddAtMostOne(alocacao[(t, p)] for t in range(len(turmas)) if horarios[t] == h)

    # Solver
    solver = cp_model.CpSolver()
    status = solver.Solve(modelo)

    # Exibir o resultado
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print('Solução encontrada:')
        for t in range(len(turmas)):
            for p in range(len(professores)):
                if solver.Value(alocacao[(t, p)]):
                    print(f'Turma {turmas[t]} com horário {horarios[t]} será alocada ao professor {professores[p]}')
    else:
        print('Nenhuma solução encontrada.')

# Dados
turmas = ['Turma A', 'Turma A', 'Turma B', 'Turma B', 'Turma C', 'Turma C', 'Turma D', 'Turma D']
horarios = ['08:00', '09:00', '10:00', '11:00', '14:00', '15:00', '08:00', '09:00']
professores = ['Prof. João', 'Prof. Maria', 'Prof. Pedro', 'Prof. Ana']

# Executar a função
alocar_professores(turmas, horarios, professores)


Solução encontrada:
Turma Turma A com horário 08:00 será alocada ao professor Prof. João
Turma Turma A com horário 09:00 será alocada ao professor Prof. João
Turma Turma B com horário 10:00 será alocada ao professor Prof. João
Turma Turma B com horário 11:00 será alocada ao professor Prof. João
Turma Turma C com horário 14:00 será alocada ao professor Prof. João
Turma Turma C com horário 15:00 será alocada ao professor Prof. João
Turma Turma D com horário 08:00 será alocada ao professor Prof. Ana
Turma Turma D com horário 09:00 será alocada ao professor Prof. Ana
